In [ ]:
#Outline
#establish connection with AWS
#check if there is a file in the scetru-ml-bucket
    #if there is a file 
       # read the file - done
       # read the do-good bucket and merge data wth the csv file (inner join) - done
       # call the model_pipeline module - done
    #after processing the file 
        # update the completed bucket
        # copy the file into archive and save file with processed_datetime
        # empty the scetru-ml-bucket

In [1]:
import sys
import os
import pandas as pd
import numpy as np
from datetime import datetime

In [3]:
#from datetime import datetime, timedelta
#sys.path.append(r'C:\\Users\\seune\\Desktop\\Projects\\Seetru-Model-Building\\')

from credentials import *
import file_handler 
import update_s3_files 

In [9]:
def ml_workflow(access_key: str, secret_access_key: str, local_folder: str):
    outcome = file_handler.ml_orchestrator(access_key, secret_access_key) # get loan decision
    complete_table =  file_handler.read_complete_table(access_key, secret_access_key) # read complete table
    complete_table = complete_table[complete_table['application_id'].isin(outcome['application_id'])] # keep the processed application
    merged_df = complete_table[['bvn','dob','amount_requested','application_id','loan_tenure','loan_repayment_structure',
                    'loan_message','internal_id', 'created_date','file_key']] \
                    .merge(outcome[['bvn', 'application_id', 'amount_approved','decline_reason']], 
                           on=['bvn', 'application_id', ], how='inner', suffixes=('', '_outcome'))
    
    merged_df['updated_date'] = pd.Timestamp.now().floor('min')
    merged_df['loan_message'] = 'COMPLETED'
    
    merged_df = merged_df[['bvn', 'dob', 'amount_requested', 'application_id', 'loan_tenure','loan_repayment_structure',
       'internal_id', 'amount_approved','created_date', 'updated_date', 'decline_reason', 'loan_message',
       'file_key']]
    
    # save the record into a local repo
    update_s3_files.saved_processed_data_as_csv(df_outcome = merged_df.drop(columns=['file_key']))
    
    # Update the completed table
    #local_folder = r'C:\\Users\\seune\\Desktop\\Projects\\Seetru-Model-Building\\development_code\\processed_loan_request'
    s3_bucket = 'complete-table'  
    s3_path_prefix = complete_table.file_key.iloc[0].split('/')[0] + '/'
    files_to_upload = list(complete_table.application_id)
    
    update_s3_files.upload_csv_files(local_folder, s3_bucket, s3_path_prefix, files_to_upload)

    # if complete_table is successfully updated do clean up 
    # Cleaning the ML bucket after processing and updating the completed tables
    test_complete_table = file_handler.S3FileHandler('complete-table', access_key, secret_access_key).check_s3_bucket_exists()
    test_complete_table['application_id'] = test_complete_table['application_id'].astype(str)
    test_complete_table = test_complete_table[test_complete_table['application_id'].isin(outcome['application_id'])]
    #if 
    #file_cleaner(access_key, secret_access_key, local_folder)

In [15]:
test_complete_table = file_handler.S3FileHandler('complete-table', access_key, secret_access_key).check_s3_bucket_exists()
test_complete_table['application_id'] = test_complete_table['application_id'].astype(str)
test_complete_table

Bucket 'complete-table' exists.


,bvn,dob,amount_requested,application_id,loan_tenure,loan_repayment_structure,loan_message,internal_id,created_date,amount_approved,decline_reason,updated_date,file_key
0,22150969042,06-05-2014,450000.0,17568395,1 year,monthly,completed,sce-38f8306d-20d7-4216-aa17-ebc2d3d3e6fd,2024-07-12 15:11,90000.0,,2024-07-12 16:18:00,fcmb_20240712/17568395.csv
1,22377535967,05-07-2018,145000.0,76887656,1 year,monthly,DECLINED,sce-bb844582-bd91-4b53-aedb-d9ed4f6f20a1,2024-07-12 16:11,NaN,Customer does not have enough transactions,NaN,fcmb_20240712/76887656.csv
2,5611253028,13-01-2015,340000.0,21753787,1 year,weekly,DECLINED,sce-08954d5f-3c1d-496a-b492-c6d8b212135e,2024-07-16 21:31,NaN,Customer does not have enough transactions,NaN,fcmb_20240716/21753787.csv
3,22150969042,03-02-2005,250000.0,17568395,1 year,monthly,Completed,sce-f67d20cb-0827-4c5e-9752-dda86bd9fb2e,2024-07-17 16:36,90000.0,,2024-07-17 17:36:00,fcmb_20240717/17568395.csv
4,22150969042,06-03-2003,345000.0,457382932,1 year,monthly,completed,sce-1b3d7604-2145-4041-9c92-f485b3dd0d70,2024-07-17 12:10,90000.0,,2024-07-17 17:18:00,fcmb_20240717/457382932.csv
5,4374739011,08-05-2001,670000.0,69879543,1 month,monthly,DECLINED,sce-5bfd964b-d1e7-4ebe-ba5b-c53e925dd7ef,2024-07-17 11:56,NaN,Customer does not have enough transactions,NaN,fcmb_20240717/69879543.csv
6,22150969042,11-03-2010,350000.0,156748766,1 year,monthly,Completed,sce-c29b2add-cedc-48bf-9303-333fd0369355,2024-07-23 10:10,90000.0,,2024-07-23 11:11:00,fcmb_20240723/156748766.csv
7,22150969042,23-07-2024,67000.0,345654566,1 year,monthly,COMPLETED,sce-3997c720-e1f8-4f45-9043-799e31270103,2024-07-23 10:25,90000.0,,2024-08-02 14:39:00,fcmb_20240723/345654566.csv
8,22377535967,30-01-2008,1200000.0,54736324,1 year,monthly,DECLINED,sce-c67d8be0-0b54-4ac8-81de-d00ce5fdd16f,2024-07-23 10:22,NaN,Customer does not have enough transactions,NaN,fcmb_20240723/54736324.csv
9,1234567890,NaN,NaN,app_id,NaN,NaN,DECLINED,NaN,NaN,NaN,Customer does not have enough transactions,NaN,fcmb_20240730/app_id.csv


In [17]:
outcome = file_handler.ml_orchestrator(access_key, secret_access_key) # get loan decision
outcome

Bucket 'scetru-ml-bucket' exists.
Bucket 'scetru-fcmb-do-good-table' exists.


,bvn,application_id,amount_requested,date_created,airtime_in_90days,bill_payment_in_90days,cable_tv_in_90days,deposit_in_90days,easy_payment_in_90days,farmer_in_90days,inter_bank_in_90days,mobile_in_90days,utility_bills_in_90days,withdrawal_in_90days,default_in_last_90days,has_it_make_it_good,amount_approved,decline_reason
0,22207845921,-1025264813,340000.0,2019-06-13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,N,Y,0.0,Loan declined due to low transaction or incomp...


In [19]:
test_complete_table = test_complete_table[test_complete_table['application_id'].isin(outcome['application_id'])]
test_complete_table

,bvn,dob,amount_requested,application_id,loan_tenure,loan_repayment_structure,loan_message,internal_id,created_date,amount_approved,decline_reason,updated_date,file_key


In [21]:
test_complete_table.loan_message == 'COMPLETED'

Series([], Name: loan_message, dtype: bool)

In [23]:
if 'COMPLETED' in test_complete_table['loan_message'].values:
    print(test_complete_table.application_id)

In [ ]:
def file_cleaner(access_key, secret_access_key, directory_to_clear):
    S3FileHandler('scetru-ml-bucket', access_key, secret_access_key).clean_ml_bucket()
    delete_files_in_directory(directory_to_clear)

In [5]:
outcome = ml_orchestrator(access_key, secret_access_key) # get loan decision
complete_table =  read_complete_table(access_key, secret_access_key) # read complete table
complete_table = complete_table[complete_table['application_id'].isin(outcome['application_id'])]
merged_df = complete_table[['bvn','dob','amount_requested','application_id','loan_tenure','loan_repayment_structure',
                        'loan_message','internal_id', 'created_date','file_key']] \
                        .merge(outcome[['bvn', 'application_id', 'amount_approved','decline_reason']], 
                               on=['bvn', 'application_id', ], how='inner', suffixes=('', '_outcome'))

merged_df['updated_date'] = pd.Timestamp.now().floor('min')
merged_df['loan_message'] = 'COMPLETED'

merged_df = merged_df[['bvn', 'dob', 'amount_requested', 'application_id', 'loan_tenure','loan_repayment_structure',
           'internal_id', 'amount_approved','created_date', 'updated_date', 'decline_reason', 'loan_message',
           'file_key']]

# save the record into a local repo
saved_processed_data_as_csv(df_outcome = merged_df.drop(columns=['file_key']))

# Upload csv files into s3 bucket
local_folder = r'C:\\Users\\seune\\Desktop\\Projects\\Seetru-Model-Building\\development_code\\processed_loan_request'
s3_bucket = 'complete-table'  
s3_path_prefix = complete_table.file_key.iloc[0].split('/')[0] + '/'
files_to_upload = list(complete_table.application_id)

upload_csv_files(local_folder, s3_bucket, s3_path_prefix, files_to_upload)

# Cleaning the ML bucket after processing and updating the completed tables
S3FileHandler('scetru-ml-bucket', access_key, secret_access_key).clean_ml_bucket()

Bucket 'scetru-ml-bucket' exists.
Bucket 'scetru-fcmb-do-good-table' exists.
Bucket 'complete-table' exists.
Files saved successfully.
Upload Successful: C:\\Users\\seune\\Desktop\\Projects\\Seetru-Model-Building\\development_code\\processed_loan_request\253435676.csv
Upload Successful: from C:\\Users\\seune\\Desktop\\Projects\\Seetru-Model-Building\\development_code\\processed_loan_request\253435676.csv into fcmb_20240802/253435676.csv
Deleted 1 .csv files from scetru-ml-bucket


In [7]:
outcome

,bvn,application_id,amount_requested,date_created,airtime_in_90days,bill_payment_in_90days,cable_tv_in_90days,deposit_in_90days,easy_payment_in_90days,farmer_in_90days,inter_bank_in_90days,mobile_in_90days,utility_bills_in_90days,withdrawal_in_90days,default_in_last_90days,has_it_make_it_good,amount_approved,decline_reason
0,22150969042,253435676,67000.0,2019-06-10,0.0,500.0,0.0,582.25,1209.0,0.0,149125.08,0.0,0.0,174325.38,N,Y,90000.0,


In [9]:
complete_table

,bvn,dob,amount_requested,application_id,loan_tenure,loan_repayment_structure,loan_message,internal_id,created_date,amount_approved,decline_reason,updated_date,file_key
0,22150969042,02-01-2008,67000.0,253435676,1 year,monthly,IN PROGRESS,sce-4d9471aa-7ea1-4234-8b2e-5313f0103e54,2024-08-02 16:26,NaN,NaN,NaN,fcmb_20240802/253435676.csv


In [11]:
merged_df

,bvn,dob,amount_requested,application_id,loan_tenure,loan_repayment_structure,internal_id,amount_approved,created_date,updated_date,decline_reason,loan_message,file_key
0,22150969042,02-01-2008,67000.0,253435676,1 year,monthly,sce-4d9471aa-7ea1-4234-8b2e-5313f0103e54,90000.0,2024-08-02 16:26,2024-08-02 17:26:00,,COMPLETED,fcmb_20240802/253435676.csv


In [9]:
# Example usage:
# processor = ApplicationProcessor('scetru-ml-bucket', access_key, secret_access_key)
# trans_data = processor.read_data()
# converted_data = processor.convert_columns(trans_data, {'amount_requested': 'float', 'date_created': 'date'})
# converted_data

In [11]:
# merged_data = processor.merge_with_do_good('scetru-fcmb-do-good-table', converted_data)
# merged_data

In [10]:
#file_reader_instance.list_csv_files()

In [11]:
#file_reader_instance.read_csv_from_s3('scetru-fcmb-do-good-table')

In [12]:
## csv_file_path = r'C:\\Users\\seune\\Downloads\\do_good_data.csv'
## df.to_csv(csv_file_path, index=False)

### Read the complete table

In [7]:
complete_table =  read_complete_table(access_key, secret_access_key)
complete_table

Bucket 'complete-table' exists.


,bvn,dob,amount_requested,application_id,loan_tenure,loan_repayment_structure,loan_message,internal_id,created_date,amount_approved,decline_reason,updated_date,file_key
0,22150969042,06-05-2014,450000.0,17568395,1 year,monthly,completed,sce-38f8306d-20d7-4216-aa17-ebc2d3d3e6fd,2024-07-12 15:11,90000.0,,2024-07-12 16:18:00,fcmb_20240712/17568395.csv
1,22377535967,05-07-2018,145000.0,76887656,1 year,monthly,DECLINED,sce-bb844582-bd91-4b53-aedb-d9ed4f6f20a1,2024-07-12 16:11,NaN,Customer does not have enough transactions,NaN,fcmb_20240712/76887656.csv
2,5611253028,13-01-2015,340000.0,21753787,1 year,weekly,DECLINED,sce-08954d5f-3c1d-496a-b492-c6d8b212135e,2024-07-16 21:31,NaN,Customer does not have enough transactions,NaN,fcmb_20240716/21753787.csv
3,22150969042,03-02-2005,250000.0,17568395,1 year,monthly,Completed,sce-f67d20cb-0827-4c5e-9752-dda86bd9fb2e,2024-07-17 16:36,90000.0,,2024-07-17 17:36:00,fcmb_20240717/17568395.csv
4,22150969042,06-03-2003,345000.0,457382932,1 year,monthly,completed,sce-1b3d7604-2145-4041-9c92-f485b3dd0d70,2024-07-17 12:10,90000.0,,2024-07-17 17:18:00,fcmb_20240717/457382932.csv
5,4374739011,08-05-2001,670000.0,69879543,1 month,monthly,DECLINED,sce-5bfd964b-d1e7-4ebe-ba5b-c53e925dd7ef,2024-07-17 11:56,NaN,Customer does not have enough transactions,NaN,fcmb_20240717/69879543.csv
6,22150969042,11-03-2010,350000.0,156748766,1 year,monthly,Completed,sce-c29b2add-cedc-48bf-9303-333fd0369355,2024-07-23 10:10,90000.0,,2024-07-23 11:11:00,fcmb_20240723/156748766.csv
7,22150969042,23-07-2024,67000.0,345654566,1 year,monthly,Completed,sce-3997c720-e1f8-4f45-9043-799e31270103,2024-07-23 10:25,90000.0,,2024-07-23 11:26:00,fcmb_20240723/345654566.csv
8,22377535967,30-01-2008,1200000.0,54736324,1 year,monthly,DECLINED,sce-c67d8be0-0b54-4ac8-81de-d00ce5fdd16f,2024-07-23 10:22,NaN,Customer does not have enough transactions,NaN,fcmb_20240723/54736324.csv
9,1234567890,NaN,NaN,app_id,NaN,NaN,DECLINED,NaN,NaN,NaN,Customer does not have enough transactions,NaN,fcmb_20240730/app_id.csv


In [ ]:
#[(complete_table['decline_reason'].isnull()) & (complete_table['amount_approved'].isnull())].reset_index(drop=True)

In [22]:
# note populate the updated_date when the 

In [9]:
outcome = ml_orchestrator(access_key, secret_access_key) # get loan decision
complete_table =  read_complete_table(access_key, secret_access_key) # read complete table
complete_table = complete_table[complete_table['application_id'].isin(outcome['application_id'])]
complete_table

Bucket 'scetru-ml-bucket' exists.
Bucket 'scetru-fcmb-do-good-table' exists.
Bucket 'complete-table' exists.


,bvn,dob,amount_requested,application_id,loan_tenure,loan_repayment_structure,loan_message,internal_id,created_date,amount_approved,decline_reason,updated_date,file_key
6,22150969042,11-03-2010,350000.0,156748766,1 year,monthly,Completed,sce-c29b2add-cedc-48bf-9303-333fd0369355,2024-07-23 10:10,90000.0,,2024-07-23 11:11:00,fcmb_20240723/156748766.csv
7,22150969042,23-07-2024,67000.0,345654566,1 year,monthly,Completed,sce-3997c720-e1f8-4f45-9043-799e31270103,2024-07-23 10:25,90000.0,,2024-07-23 11:26:00,fcmb_20240723/345654566.csv


In [23]:
# if read_complete_table return zero file, it means the record in scetru ml bucket has already being processed.

In [10]:
outcome = ml_orchestrator(access_key, secret_access_key) # get loan decision
complete_table =  read_complete_table(access_key, secret_access_key) # read complete table
complete_table = complete_table[complete_table['application_id'].isin(outcome['application_id'])]
merged_df = complete_table[['bvn','dob','amount_requested','application_id','loan_tenure','loan_repayment_structure',
                        'loan_message','internal_id', 'created_date','file_key']] \
                        .merge(outcome[['bvn', 'application_id', 'amount_approved','decline_reason']], 
                               on=['bvn', 'application_id', ], how='inner', suffixes=('', '_outcome'))

merged_df['updated_date'] = pd.Timestamp.now().floor('min')
merged_df['loan_message'] = 'COMPLETED'

merged_df = merged_df[['bvn', 'dob', 'amount_requested', 'application_id', 'loan_tenure','loan_repayment_structure',
           'internal_id', 'amount_approved','created_date', 'updated_date', 'decline_reason', 'loan_message',
           'file_key']]

Bucket 'scetru-ml-bucket' exists.
Bucket 'scetru-fcmb-do-good-table' exists.
Bucket 'complete-table' exists.


In [13]:
merged_df

,bvn,dob,amount_requested,application_id,loan_tenure,loan_repayment_structure,internal_id,amount_approved,created_date,updated_date,decline_reason,loan_message,file_key
0,22150969042,11-03-2010,350000.0,156748766,1 year,monthly,sce-c29b2add-cedc-48bf-9303-333fd0369355,90000.0,2024-07-23 10:10,2024-08-02 14:36:00,,COMPLETED,fcmb_20240723/156748766.csv
1,22150969042,23-07-2024,67000.0,345654566,1 year,monthly,sce-3997c720-e1f8-4f45-9043-799e31270103,90000.0,2024-07-23 10:25,2024-08-02 14:36:00,,COMPLETED,fcmb_20240723/345654566.csv


### Update the complete table

In [12]:
# save the csv file into dir - processed_loan_request
saved_processed_data_as_csv(df_outcome = merged_df.drop(columns=['file_key']))

Files saved successfully.


In [15]:
# Replace these with your actual values
# Upload csv files into s3 bucket
local_folder = r'C:\\Users\\seune\\Desktop\\Projects\\Seetru-Model-Building\\development_code\\processed_loan_request'
s3_bucket = 'complete-table'  
s3_path_prefix = complete_table.file_key.iloc[0].split('/')[0] + '/'
files_to_upload = list(complete_table.application_id)

upload_csv_files(local_folder, s3_bucket, s3_path_prefix, files_to_upload)

Upload Successful: C:\\Users\\seune\\Desktop\\Projects\\Seetru-Model-Building\\development_code\\processed_loan_request\345654566.csv
Upload Successful: from C:\\Users\\seune\\Desktop\\Projects\\Seetru-Model-Building\\development_code\\processed_loan_request\345654566.csv into fcmb_20240723/345654566.csv


In [ ]:
# delete items from scetru-ml-bucket after processing - store up application-id list. if id in a processed list then delete
# delete the local folder *after every run

In [17]:
# Cleaning the ML bucket after processing and updating the completed tables
S3FileHandler('scetru-ml-bucket', access_key, secret_access_key).clean_ml_bucket()

Deleted 2 .csv files from scetru-ml-bucket


In [16]:
import os
import shutil

def delete_files_in_directory(directory_path):
  """Deletes all files in a given directory.

  Args:
    directory_path: The path to the directory to be cleared.
  """

  for file in os.listdir(directory_path):
    file_path = os.path.join(directory_path, file)
    try:
      if os.path.isfile(file_path):
        os.remove(file_path)
    except OSError as e:
      print(f"Error deleting file {file_path}: {e}")

In [20]:
# Example usage:
directory_to_clear = r'C:\\Users\\seune\\Desktop\\Projects\\Seetru-Model-Building\\development_code\\processed_loan_request'
delete_files_in_directory(directory_to_clear)

In [22]:
S3FileHandler('scetru-ml-bucket', access_key, secret_access_key).clean_ml_bucket()

No .csv files found in the bucket


In [50]:
import boto3

# Initialize a session using Amazon S3

bucket_name = bucket_name
session = boto3.Session(
aws_access_key_id=access_key,
aws_secret_access_key=secret_access_key)
s3 = session.client('s3')

# List all the objects in the bucket
response = s3.list_objects_v2(Bucket='scetru-ml-bucket')

if 'Contents' in response:
    # Filter for objects that end with .csv
    files_to_delete = [obj['Key'] for obj in response['Contents'] if obj['Key'].endswith('.csv')]
files_to_delete

['fcmb_20240701/trn-14343437.csv']

In [85]:
def clean_ml_bucket(bucket_name):
    
    files_to_delete = S3FileHandler(bucket_name, access_key, secret_access_key).list_csv_files()

    # Delete the csv files
    if files_to_delete:
        delete_objects = {'Objects': [{'Key': key} for key in files_to_delete]}
        s3.delete_objects(Bucket=bucket_name, Delete=delete_objects)
        print(f"Deleted {len(csv_files)} .csv files from {bucket_name}")
    else:
        print("No .csv files found in the bucket")

In [87]:
clean_ml_bucket('scetru-ml-bucket')

NameError: name 's3' is not defined

In [3]:
objects

s3.Bucket.objectsCollection(s3.Bucket(name='transaction-hist'), s3.ObjectSummary)

In [12]:
import boto3
from secrets import access_key, secret_access_key

s3_client = boto3.client('s3',
                     aws_access_key_id = access_key,
                     aws_secret_access_key = secret_access_key)
bucket_name = "transaction-hist"
result = s3_client.list_objects(Bucket = bucket_name, Delimiter = '/')
##result_a = result['Contents']

out = []
for obj in result["Contents"]:
    search_key = 'Key'
    res = [val for key, val in obj.items() if search_key in key]
    out.append(res)
p = {}
for i in range(len(out)):
    p[i] = out[i]

p
    

{0: ['22150969042.csv'],
 1: ['22170265087.csv'],
 2: ['22230658590.csv'],
 3: ['22230658598.csv'],
 4: ['22327321259.csv'],
 5: ['22381611055.csv'],
 6: ['22381849810.csv']}

In [13]:
accesskey = access_key
secretkey = secret_access_key
region = "us-east-1"

s3_resource = boto3.resource ('s3', aws_access_key_id = accesskey, aws_secret_access_key = secretkey, region_name = region)
s3_resource.meta.client.delete_objects(Bucket='transaction-hist', Delete={'Objects': p})

ParamValidationError: Parameter validation failed:
Invalid type for parameter Delete.Objects, value: {0: ['22150969042.csv'], 1: ['22170265087.csv'], 2: ['22230658590.csv'], 3: ['22230658598.csv'], 4: ['22327321259.csv'], 5: ['22381611055.csv'], 6: ['22381849810.csv']}, type: <class 'dict'>, valid types: <class 'list'>, <class 'tuple'>